In [ ]:
import importlib, os, pandas, re
os.chdir('/Users/dfporter/pma/dataAndScripts/clip/miseq/v2all/')

import sameRiver
import sameRiver.metadata
import sameRiver.metadata.negative_metadata as negative_metadata
import sameRiver.metadata.positive_metadata_v2 as positive_metadata
import sameRiver.negativeCounts
import sameRiver.positiveCounts
import sameRiver.scheme
import sameRiver.statsForCountsNB
#importlib.reload(sameRiver.countsO)
importlib.reload(sameRiver.statsForCountsNB)
importlib.reload(sameRiver.scheme)

def normalizations(use_bed_dir):

    negatives = sameRiver.negativeCounts.negativeCounts(negative_metadata)
    negatives.normalize_ann_counts(use_bed_dir=use_bed_dir)
    negatives.get_lowest_values_for_all_proteins()
    # Optional: write_txt=True to write some txt's of the data.
    negatives.save(write_object=True, write_txt=False)
    
    positives = sameRiver.positiveCounts.positiveCounts(positive_metadata)
    positives.normalize_ann_counts(use_bed_dir=use_bed_dir)
    positives.save(write_object=True, write_txt=False)

    return negatives, positives

def evaluate_metrics(pval_source, per_what, pvals_file):
    ba = sameRiver.biotypeAdder.biotypeAdder.load()
    lines = []
    lines.append("#" * 14)
    lines.append([pval_source, per_what, pvals_file])
    pvals = pandas.read_excel(pvals_file, index_col=0)
    
    pvals= ba.add_biotypes_column_from_gene_name(pvals)
    pvals['Gene type'] = [
        {True: 'Repetitive element', False: _t}[bool(re.search('.*/.*', name))] for \
        name, _t in zip(pvals.index, pvals['Gene type'])]
    lines.append(pvals.shape)
    
    for cutoff in [0.05, 1E-10]:
        lines.append('-----P<{}-----'.format(cutoff))
        for protein in ['Rbfox2', 'hnRNPC',  'PCBP1', 'ETS2', 'UBA2', 'CCIN', 'IDE', 'EPB41L5']:
            lines.append(protein)
            targets = pvals[pvals[protein]<cutoff].copy()
            lines.append(targets.shape)
            lines.append(targets['Gene type'].value_counts().head(5))

    lines = '\n'.join([str(x) for x in lines])
    print(lines)
def try_all_params():
    
    #for use_bed_dir in [None, './beds/']:
        
    negatives = sameRiver.negativeCounts.negativeCounts.load()
    positives = sameRiver.positiveCounts.positiveCounts.load()

    #negatives, positives = normalizations(use_bed_dir)

    nb = sameRiver.statsForCountsNB.statsForCountsNB(
        negatives=negatives, positives=positives)

    nb = sameRiver.statsForCountsNB.statsForCountsNB.load()
    #for pval_source in ['by_rna', 'by_location', 'by_both']:

    #    if pval_source == 'by_rna':
    pval_source = 'by_rna'
    #nb.calculate_pvalues(which='per_read', apply_bh_adjust=True)
    #nb.write_pvals_single_file(which='per_read', outfname='./tables/pvals_per_read.xlsx')
    evaluate_metrics(pval_source, 'per_read', './tables/pvals_per_read.xlsx')

    #nb.calculate_pvalues(which='per_protein', apply_bh_adjust=True)
    #nb.write_pvals_single_file(which='per_protein', outfname='./tables/pvals_per_protein.xlsx')
    evaluate_metrics(pval_source, 'per_protein', './tables/pvals_per_protein.xlsx')

    nb.save()
try_all_params()
